In [274]:
# import sqlite3
# conn = sqlite3.connect('sales_sql')
import importlib
import tqdm
importlib.reload(tqdm)
from tqdm import tqdm
import time
import pickle

import requests
from tqdm import tqdm


In [61]:
categories = ['Информация', 'Общество', 'Природа', 'Техника', 'Человек', 
 'Транспорт', 'Спорт', ' Политика', 'Культура', 'Происшествия', 'Экономика', 'Наука']

In [219]:
S = requests.Session()
URL = "https://ru.wikipedia.org/w/api.php"

In [150]:
def get_subcategories(category):
    PARAMS = {
        "action": "query",
        "cmtitle": "Category:%s" % category,
        "cmtype": "subcat",
        "prop": "categoryinfo",
        "cmlimit": "max",
        "list": "categorymembers",
        "format": "json"
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA['query']['categorymembers']

In [217]:
def get_category_pages(category):
    PARAMS = {
        "action": "query",
        "cmtitle": "Category:%s" % category,
        "cmtype": "page",
        "prop": "categoryinfo",
        "cmnamespace": "0",
        "cmlimit": "max",
        "list": "categorymembers",
        "format": "json"
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA['query']['categorymembers']

### Строим иерархию категорий Википедии глубиной 5 уровней.

In [192]:
category_dict = {} # pageid - title
pages_dict = {} # pageid - title
children = {} # pageid - [pageid]
queue = []
visited = set()
levels = {}

for category in categories:
    queue.append((category, category))
    levels[category] = 0
    visited.add(category)

cnt = 0
while queue:
    cnt += 1
    curr_page_id, curr_category_name = queue.pop(0)
    
    if cnt % 200 == 0:
        print(cnt, levels[curr_page_id], curr_page_id, curr_category_name)
    if levels[curr_page_id] > 6:
        break
    for result in get_subcategories(curr_category_name):
        if result['pageid'] not in visited:
            category_dict[result['pageid']] = result
            if levels[curr_page_id] == 0:
                print(cnt, curr_page_id, '   ', result['title'][10:])
            queue.append((result['pageid'], result['title'][10:]))
            visited.add(result['pageid'])
            levels[result['pageid']] = levels[curr_page_id] + 1
            if curr_page_id not in children:
                children[curr_page_id] = []
            children[curr_page_id].append(result['pageid'])


1 Информация     Асимметричность информации
1 Информация     Визуализация
1 Информация     Визуализация информации
1 Информация     Данные
1 Информация     Знаковые системы
1 Информация     Знание
1 Информация     Информационная наука
1 Информация     Информационное общество
1 Информация     Информационное право
1 Информация     Информационные технологии
1 Информация     Источники информации
1 Информация     Носители информации
1 Информация     Ошибки
1 Информация     Персоналии:Информация
1 Информация     Рекорды
1 Информация     Связь
1 Информация     Секретная информация
1 Информация     Символы
1 Информация     Стандартизация
1 Информация     Статистика
1 Информация     Статистические данные
1 Информация     Структура
1 Информация     Текст
2 Общество     Википедия:Избранные статьи о культуре и обществе
2 Общество     Общественные науки
2 Общество     Общество по континентам
2 Общество     Общество по странам
2 Общество     Списки:Общество
2 Общество     Безопасность
2 Общество    

12 Наука     Научные дисциплины
12 Наука     Текущие события в науке
12 Наука     Наука по местоположению
12 Наука     Портал:Наука
12 Наука     Списки:Наука
12 Наука     Научные базы данных
12 Наука     Время в науках
12 Наука     Научные сооружения
12 Наука     Научные исследования
12 Наука     История науки
12 Наука     Научные коллективы
12 Наука     Научные концепции
12 Наука     Научная литература
12 Наука     Маргинальная наука
12 Наука     Научный метод
12 Наука     Науковедение
12 Наука     Наукометрия
12 Наука     Незавершённые статьи о науке
12 Наука     Об учёных
12 Наука     Научное оборудование
12 Наука     Наука и общество
12 Наука     Открытая наука
12 Наука     Персоналии:Наука
12 Наука     Научные понятия
12 Наука     Научные проблемы
12 Наука     Стипендии
12 Наука     Философия науки
12 Наука     Научные школы
12 Наука     Википедия:Избранные статьи по наукам
200 1 7732121 Идеология
400 2 242116 Софистика
600 2 4076591 Общество Дании
800 2 1777435 Природа Южной Амер

36600 4 5067160 Министры окружающей среды Канады
36800 4 7444907 Системная медицина
37000 4 1405308 Регионы Азербайджана
37200 4 32168 Федеративные Штаты Микронезии
37400 4 7257240 Диссиденты по странам
37600 4 1538761 Наркоторговцы
37800 4 2382849 Незавершённые статьи по малакологии
38000 4 1302121 Незавершённые статьи по географии Боливии
38200 4 1529986 Незавершённые статьи о Вооружённых силах России
38400 4 5481724 Эндемики Центральной Африки
38600 4 803441 Вулканы США
38800 4 4817815 Озёра Черниговской области
39000 4 743592 Заказники Белоруссии
39200 4 3734079 Охраняемые природные территории Иркутской области
39400 4 3397252 Парки развлечений в Японии
39600 4 144360 События июля
39800 4 50346 Категории по океанам
40000 4 1436605 Горы Малайзии
40200 4 3412977 Мысы Канады
40400 4 42004 Атолл Джонстон
40600 4 298490 Полуострова Казахстана
40800 4 1723254 Персонажи-инопланетяне
41000 4 142535 Геодинамические обстановки
41200 4 2883775 Астрономические объекты, открытые в 1868 году
414

74400 4 1760154 Административное деление Швейцарии
74600 4 5175793 Парламент Венесуэлы
74800 4 4945888 Министерства Португалии
75000 4 5022749 Женщины-политики России
75200 4 2157373 Коммунисты Кипра
75400 4 189904 Революционеры Перу
75600 4 1820149 Президенты Боснии и Герцеговины
75800 4 7907958 Члены Исследовательского комитета по подготовке индонезийской независимости
76000 4 3582850 Главы городов Монголии
76200 4 6418556 Члены партий Турции
76400 4 7780081 Политический сыск в Австрии
76600 4 7473476 Законодательное собрание Альберты
76800 4 8007700 Коммунистические партии Туниса
77000 4 4961323 Территориальные споры
77200 4 3375972 Административные единицы по уровню
77400 4 4441816 Народное образование в Третьем рейхе
77600 4 2250452 Изображения:Листовки
77800 4 3801370 Супруги правителей по именам
78000 4 4787825 Деятели культуры Венесуэлы
78200 4 4794359 Переводчики Бельгии
78400 4 5333965 Персоналии:Псково-Печерский монастырь
78600 4 6207268 Костюм по векам
78800 4 1494701 1957 

KeyboardInterrupt: 

In [193]:
len(levels)

196514

In [194]:
with open('./wiki_category_dict.pkl', 'wb') as f:
    pickle.dump(category_dict, f)
    
with open('./wiki_pages_dict.pkl', 'wb') as f:
    pickle.dump(pages_dict, f)

with open('./wiki_children.pkl', 'wb') as f:
    pickle.dump(children, f)
    
with open('./wiki_levels.pkl', 'wb') as f:
    pickle.dump(levels, f)

with open('./wiki_queue.pkl', 'wb') as f:
    pickle.dump(queue, f)
    
with open('./wiki_visited.pkl', 'wb') as f:
    pickle.dump(visited, f)
    


In [195]:
cnt

101345

In [196]:
len(category_dict)

196502

In [197]:
cat_names = [category_dict[x]['title'] for x in category_dict]

In [ ]:
cat_names = [category_dict[x]['title'] for x in category_dict]
print('Категория:Транспорт' in cat_names)

for x in category_dict:
    if category_dict[x]['title'] == 'Категория:Транспорт':
        print(x)

In [165]:
category_dict[6424459]

{'ns': 14, 'pageid': 6424459, 'title': 'Категория:Экономика по отраслям'}

### Для каждой категории получаем список статей 

In [265]:
# pages_dict = {}
for i, category in enumerate(categories_not_proc):
    category_info = category_dict[category]
    
    if i % 1000 == 0:
        print(i, category_info)

    if category in categories_proc:
        continue
    for result in get_category_pages(category_info['title'][10:]):
        if result['pageid'] not in pages_dict:
            result['parent'] = [category]
            pages_dict[result['pageid']] = result
        else:
            pages_dict[result['pageid']]['parent'].append(category)

0 {'pageid': 6815744, 'ns': 14, 'title': 'Категория:Керамика Греции'}
1000 {'pageid': 7341596, 'ns': 14, 'title': 'Категория:Наука в Абруцци'}
2000 {'pageid': 1838123, 'ns': 14, 'title': 'Категория:Животные, описанные в 1848 году'}
3000 {'pageid': 4985404, 'ns': 14, 'title': 'Категория:Чикаго Американ Гиэрс'}
4000 {'pageid': 268395, 'ns': 14, 'title': 'Категория:Изображения:Кометы'}
5000 {'pageid': 5512841, 'ns': 14, 'title': 'Категория:Категории кинокомпаний СССР'}
6000 {'pageid': 3941376, 'ns': 14, 'title': 'Категория:Археологи Малайзии'}
7000 {'pageid': 4074049, 'ns': 14, 'title': 'Категория:Хоккеисты на траве Германии'}
8000 {'pageid': 4599936, 'ns': 14, 'title': 'Категория:Транспортные происшествия в Антарктиде'}
9000 {'pageid': 3815139, 'ns': 14, 'title': 'Категория:Скульпторы Мексики'}
10000 {'pageid': 5652013, 'ns': 14, 'title': 'Категория:Армянский эмират'}
11000 {'pageid': 7095510, 'ns': 14, 'title': 'Категория:Председатели Президиума Верховного Совета Марийской АССР'}
12000 

In [245]:
category_dict[11303], category_dict[8688]

({'ns': 14, 'pageid': 11303, 'title': 'Категория:Информационные технологии'},
 {'ns': 14, 'pageid': 8688, 'title': 'Категория:Связь'})

### Для именованных сущностей определяем верхнюю категорию в иерархии Wiki.

In [276]:
import pandas as pd
import wikipedia
import Levenshtein
import re

wikipedia.set_lang("ru")

In [280]:
locs = pd.read_csv('./locs.txt', header=-1)
locs.columns = ['value']
pers = pd.read_csv('./pers.txt', header=-1, sep='\n')
pers.columns = ['value']
orgs = pd.read_csv('./orgs.txt', header=-1, sep=';')
orgs.columns = ['value']

# orgs = pd.read_csv('./orgs.csv', index_col=0)
# pers = pd.read_csv('./pers.csv', index_col=0)
# locs = pd.read_csv('./locs.csv', index_col=0)

Определяем для каждой категории верхнего уровня список всех подкатегорий

In [ ]:
cat2subcats = {}
for category in categories:
    print(category)
    
    if category not in cat2subcats:
        cat2subcats[category] = []
    queue = children[category]
    
    while queue:
        curr_id = queue.pop(0)
        cat2subcats[category].append(curr_id)
        if curr_id in children:
            queue.extend(children[curr_id])

Инвертируем: для каждой категории определяем верхнеуровневую категорию

In [ ]:
cat2toplevel = {}
for key in cat2subcats:
    for catid in cat2subcats[key]:
        if catid not in cat2toplevel:
            cat2toplevel[catid] = (key)

In [ ]:
page_title2toplevel = {}
for pageid in tqdm(pages_dict):
    for catid in pages_dict[pageid]['parent']:
        pagetitle = pages_dict[pageid]['title']
        if pagetitle not in page_title2toplevel:
            page_title2toplevel[pagetitle] = []
        if catid in cat2toplevel:
            page_title2toplevel[pagetitle].append(cat2toplevel[catid])

In [ ]:
res = []
for val in tqdm(locs['value']):
    try:
        page_name = wikipedia.search(val)[0]
    except:
        time.sleep(1)
        page_name = None
    res.append(page_name)
    
locs['wiki_page'] = res
locs['top_level'] = locs['wiki_page'].apply(lambda x: ';'.join(set(page_title2toplevel[x])) 
                                            if x in page_title2toplevel else None)

In [320]:
def levenshtein_dist(query, candidate):
    candidate_clean = re.sub(r'\([^)]*\)', '', candidate).strip()
    leven = Levenshtein.distance(query.lower(), candidate_clean.lower())
    return leven

def Jaccard(str1, str2):
    str1 = re.sub(r'\([^)]*\)', '', str1).strip().lower()
    str2 = re.sub(r'\([^)]*\)', '', str2).strip().lower()
    parts1 = [x for x in re.findall(r'(\w*)', str1) if x != '']
    parts2 = [x for x in re.findall(r'(\w*)', str2) if x != '']
#     candidate_parts = [x for x in candidate_parts if x != '']
    return len(set(parts1) & set(parts2)) / len(set(parts1) | set(parts2))

In [323]:
wiki_names = []
l_dists = []
for i, val in enumerate(locs['value']):
    try:
        page_name = wikipedia.search(val)[0]
        l_dist = levenshtein_dist(val, page_name)
    except:
        time.sleep(1)
        page_name = None
        l_dist = None
    wiki_names.append(page_name)
    l_dists.append(l_dist)
    if i % 100 == 0:
        print(i, val, page_name, l_dist)

0 солнцукогда None None
100 британский колумбия Британская Колумбия 2
200 нижний тогузак Нижний Тогузак 0
300 эт-танфа Эт-Танф 1
400 фото Фотография 6
500 северный корея Корейская Народно-Демократическая Республика 37
600 таганрог Таганрог 0
700 вирджиния Виргиния 2
800 старицкий Старицкий, Михаил Петрович 17
900 верхний лихобора Верхние Лихоборы (станция метро) 2
1000 ниагарский водопад Ниагарский водопад 0
1100 дальний восток Дальний Восток 0
1200 сагб Сагбо 1
1300 ново-солдатский Полки иноземного строя 19
1400 ульяновский область Ульяновская область 2
1500 саудовский аравия Саудовская Аравия 2
1600 норт-ласа-вегас Лас-Вегас 6
1700 майя Майя (цивилизация) 0
1800 магриб Магриб 0
1900 аубари Аубари 0
2000 свиблово Свиблово (район Москвы) 0
2100 пятигорск Пятигорск 0
2200 восточный фронт Восточный фронт 0
2300 северский донец Северский Донец 0
2400 ковалёвка Ковалевка 1
2500 сырдарья Сырдарья 0
2600 боливия Боливия 0
2700 покровка Покровка 0
2800 оренбуржье Оренбуржье (авиакомпания) 0


In [363]:
locs['levenshtein'] = locs.apply(lambda x: levenshtein_dist(x['value'], x['wiki_page']) 
                                 if isinstance(x['wiki_page'], str) else None, axis=1)
locs['levenshtein_normed'] = locs.apply(lambda x: x['levenshtein']  / len(x['value']), axis=1)
locs['intersect'] = locs.apply(lambda x: Jaccard(x['value'], x['wiki_page'])
                                         if isinstance(x['wiki_page'], str) else None, axis=1)

In [365]:
pers['levenshtein'] = pers.apply(lambda x: levenshtein_dist(x['value'], x['wiki_page']) 
                                 if isinstance(x['wiki_page'], str) else None, axis=1)
pers['levenshtein_normed'] = pers.apply(lambda x: x['levenshtein']  / len(x['value']), axis=1)
pers['intersect'] = pers.apply(lambda x: Jaccard(x['value'], x['wiki_page'])
                                         if isinstance(x['wiki_page'], str) else None, axis=1)

In [366]:
orgs['levenshtein'] = orgs.apply(lambda x: levenshtein_dist(x['value'], x['wiki_page']) 
                                 if isinstance(x['wiki_page'], str) else None, axis=1)
orgs['levenshtein_normed'] = orgs.apply(lambda x: x['levenshtein']  / len(x['value']), axis=1)
orgs['intersect'] = orgs.apply(lambda x: Jaccard(x['value'], x['wiki_page'])
                                         if isinstance(x['wiki_page'], str) else None, axis=1)

In [367]:
orgs[orgs['levenshtein_normed'] > 0.6]

,value,wiki_page,top_level,levenshtein,levenshtein_normed,intersect
1,сирийский красный полумесяц,Международное движение Красного Креста и Красн...,Информация,40.0,1.481481,0.000000
4,одесский облгосадминистрация,Противостояние в Одессе (2013—2014),Общество,25.0,0.892857,0.000000
5,клипперс,Лос-Анджелес Клипперс,Спорт,13.0,1.625000,0.333333
7,студенческий спортивный лига,Московская студенческая хоккейная лига,Спорт,21.0,0.750000,0.166667
12,спортивно-дисциплинарный комитет,Континентальная хоккейная лига,Спорт,27.0,0.843750,0.000000
13,христианский государство – святой русь,Святая Русь,Общество;Природа;Культура; Политика;Информация,29.0,0.763158,0.200000
16,эдмонт,"Тантыба, Эдмонт Акифович",NaN,18.0,3.000000,0.333333
18,краматорский пограничный отряд,Улицы Краматорска,Транспорт,25.0,0.833333,0.000000
21,the daily star,The Black Eyed Peas,Культура;Природа;Техника,11.0,0.785714,0.166667
22,harris interactive and harris poll,Любимая архитектура Америки,Культура;Общество,33.0,0.970588,0.000000


In [368]:
locs.to_csv('locs.csv')
pers.to_csv('pers.csv')
orgs.to_csv('orgs.csv')